### Ideen Use Case
- Gesetzbuch ZGB
- Kochbuch
- LLM Agent generiert Werbetexe, Social Media Posts

### FWO-Assistent

- Hilft bei der Erstellung von Texten für LinkedIn, Instagram, WhatsApp etc.

- Auf Basis von vergangenen Posts

In [1]:
import os
from langchain_openai import ChatOpenAI

# --- Stelle sicher, dass dieser Key existiert ---
assert "GROQ_API_KEY" in os.environ, "GROQ_API_KEY fehlt in den Env Vars!"

# --- Initialisiere LLM explizit für OpenRouter ---
llm = ChatOpenAI(
    model="openai/gpt-oss-120b",   # wähle ein OpenRouter-Modell, das es wirklich gibt
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1",
    temperature=0.3,
)

print("Sende Test-Ping...")
try:
    msg = llm.invoke("Sag exakt: pong")
    print("Antworttyp:", type(msg))
    # msg ist i.d.R. ein AIMessage – gib Inhalt sicher aus:
    print("Inhalt:", getattr(msg, "content", msg))
except Exception as e:
    print("FEHLER beim LLM-Aufruf:", repr(e))


c:\Users\amuel\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sende Test-Ping...
Antworttyp: <class 'langchain_core.messages.ai.AIMessage'>
Inhalt: pong


In [2]:
import os
import langchain
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI


#LLM_MODEL = "openai/gpt-oss-20b:free"
LLM_MODEL = "openai/gpt-oss-120b"
LLM_TEMPERATURE = 0.4
BASE_URL = "https://api.groq.com/openai/v1"
OPENROUTER_API_KEY = os.getenv("GROQ_API_KEY")
USER_PROMPT="Generiere mir einen Instagrampost"

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    base_url=BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

print(type(llm))


<class 'langchain_openai.chat_models.base.ChatOpenAI'>


In [4]:
try:
    print(llm.invoke("Sag nur: pong").content)
except Exception as e:
    print(repr(e))

# Test 2: Env-Variablen sichtbar?
import os
print("OPENAI_API_KEY" in os.environ, os.environ.get("OPENAI_BASE_URL"))
print("GROQ_API_KEY" in os.environ)
print("OPENROUTER_API_KEY" in os.environ)

pong
False None
True
False


In [5]:
from langchain_core.prompts import ChatPromptTemplate

FWO_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "Du bist der FWO-Assistent (Fachschaft Wirtschaft Olten, FHNW). "
     "Bevor du schreibst, beziehst du dich IMMER auf: "
     "1) Kampagnenkonzept (Ziel, Kernbotschaft, Mehrwert) "
     "2) Redaktionsplan (Phase, wiederkehrende Formate, Timing). "
     "Vermeide Genderung und gebrauche bspw. Lehrperson anstatt Lehrer*innen "
     "Generiere jeweils immer zwei Vorschläge zu jeder Plattform (Instagram, WhatsApp, LinkedIn)"
     "Vermeide Floskeln, schreibe aktiv und konkret."
     "If you are unsure or the answer isn't in the context, say that you don't know.\n\n"
     "CONTEXT: \n {context}"
    ),
    ("human","{question}"),
])


In [ ]:
# from langchain_core.output_parsers import StrOutputParser

# user_prompt = "Generiere mir einen Post für die PubTour am 16. Oktober"

# chain_simple = (FWO_PROMPT | llm | StrOutputParser())
# result = chain_simple.invoke({"question": user_prompt, "context": ""})
# print(result)
# print(result)

**Kampagnenkonzept**  
- **Ziel:** Studierende und Interessierte zur PubTour am 16. Oktober motivieren und die FHNW als lebendigen Netzwerk‑ und Lernort präsentieren.  
- **Kernbotschaft:** „Entdecke, vernetze, feiere – die PubTour ist dein Sprungbrett für Praxis, Kontakte und Spaß.“  
- **Mehrwert:** Direkter Einblick in aktuelle Projekte, Möglichkeit zum Austausch mit Lehrpersonen und Unternehmen, lockere Atmosphäre zum Knüpfen von Kontakten.

**Redaktionsplan**  
- **Phase:** Awareness → Reminder (2 Wochen vor, 3 Tage vorher, Tag‑der‑Tour).  
- **Wiederkehrende Formate:** Teaser‑Grafik/Video, Countdown‑Story, Kurz‑FAQ.  
- **Timing:** Instagram – Montag 09:00 / Freitag 16:00, WhatsApp – Dienstag 12:00 / Donnerstag 18:00, LinkedIn – Mittwoch 08:30 / Freitag 14:00.  

---

## Instagram  

### Vorschlag 1 (Teaser‑Grafik, Montag 09:00)  
**Bild:** Foto einer belebten Pub‑Location mit FHNW‑Logo im Vordergrund.  
**Text:**  
> 🎉 PubTour 16. Oktober – Dein Einstieg ins Netzwerk!  
> • Prax

In [7]:
from langchain_community.document_loaders import PyPDFLoader

# List of PDF file paths

pdf_path = "Order pdf/SoMe Konzept.pdf"

loader = PyPDFLoader(str(pdf_path))
pages = loader.load()


# Load all documents
# all_pages_pdf = []
# for pdf in pdf_files:
#     loader = PyPDFLoader(pdf)
#     pages = loader.load()
#     all_pages_pdf.extend(pages)

#print(f"Loaded {len(all_pages_pdf)} pages from {len(pdf_files)} pdf documents.")


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# get both websites and pdfs together
all_docs = pages

# define the splitter and strategy
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = splitter.split_documents(all_docs)

In [9]:
import numpy as np

lengths = [len(s.page_content) for s in splits]
print(f"Initial documents: {len(all_docs)}")
print(f"Total chunks: {len(splits)}")
print(f"Avg length: {np.mean(lengths):.1f}")
print(f"Min: {np.min(lengths)}, Max: {np.max(lengths)}")

Initial documents: 6
Total chunks: 55
Avg length: 253.0
Min: 48, Max: 297


In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


C:\Users\amuel\AppData\Local\Temp\ipykernel_26292\4293025327.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [11]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    normalize_L2=True
)

vector_store.add_documents(documents=splits)

['0ec7a176-a859-440f-9aea-512c66d0c330',
 '482ee17f-3b27-4af4-9572-c0c6c904bb28',
 '6bde600b-589b-4cf2-814e-9bf0a2f1798e',
 'deeea46f-9cc2-4490-8110-29cc924fc92c',
 '7dc1ba28-eee3-4689-8895-7fe756db4e3f',
 'c9decb82-0985-4721-81a2-821808f76e60',
 'b9407149-31ac-495f-83d1-ddc514feb0d2',
 '968e163d-0e13-4620-b62d-75a81cc6b9c1',
 'd4efc9da-60df-4d4e-8b2e-cec5ebdd1fc0',
 '99dbc341-fbbf-4edb-ad62-f3582631b7d3',
 '5fbeb404-3d6d-4645-8db9-a85577a6b370',
 '28604f02-79f6-4d9b-b205-d84e5cd51e50',
 'afd49e8b-1a90-41fb-a324-bb2643ea5161',
 '7e16ee91-33fb-4fd9-854d-30b6d065ae1a',
 '7b3cd5cc-3725-4474-9ce6-ee98f0583b71',
 '183a3be6-4d4d-4260-93ba-f829560bd1c2',
 '36e4ecd6-9fb3-43ad-b49f-065729b9f12f',
 'cfd2e891-c272-4e4e-ae7a-48b7c8525b96',
 '280e8ec0-01db-4e77-b5d8-4dea9f65dcd0',
 '442d71a3-64fc-4041-9b53-63766aa168f2',
 'e3067aa3-5088-4c9b-a030-bbf0c57fa19f',
 '1b91d4c7-307d-4090-a35f-b61a8eacb99e',
 '080e9581-3201-4f74-a6be-5030f01ed2ed',
 '82f81bd6-c092-42eb-a0ce-d9746a84c9b0',
 '4a1b2a97-ef2a-

In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [13]:
docs = retriever.get_relevant_documents("Hashtags für Instagram")
for i, d in enumerate(docs, 1):
    print(f"--- Retrieved doc {i} ---")
    print(d.page_content[:400], "...\n")

--- Retrieved doc 1 ---
- Grund: Account aktiv halten vor allem in Zeiten, bei denen weniger Posts gemacht 
werden können. 
Hashtags: 
#FHNW #FWO #FachschaftWirtschaftOlten #HappyEaster #HappyChristmas #HappyXmas und 
Spezifische Hashtags die von Partnern vorgegeben werden. 
Standard Seeding: ...

--- Retrieved doc 2 ---
Hashtags: Keine Hashtags 
Seeding: kein Seeding. ...

--- Retrieved doc 3 ---
Die meisten Jugendlichen/jungen Erwachsene sind auf Instagram aktiv. Das Posten von Events 
auf diesem Kanal ermöglicht der FWO ein gutes und interessantes Image für die FHNW in Olten 
zu erschaffen und somit zukünftige Studierende zu „überzeugen“ an der FH in Olten studieren zu ...



C:\Users\amuel\AppData\Local\Temp\ipykernel_26292\3188004710.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Hashtags für Instagram")


In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | FWO_PROMPT
#     | llm
#     | output_parser
# )

chain = (
{
    "context": retriever,
    "question": RunnablePassthrough(),
}
    | FWO_PROMPT
    | llm
    | StrOutputParser()
)


In [15]:
# from langchain_core.output_parsers import StrOutputParser

# USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

# result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
# print(result)

result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
print(result)

#result = chain.invoke(user_prompt)
#print(result)

**Kampagnenkonzept (PubTour 2025)**  
- **Ziel:** Schnelle, direkte Kommunikation der Event‑Infos und Steigerung der Teilnahme.  
- **Kernbotschaft:** „Gemeinsam lernen, netzwerken und feiern – die PubTour am 16. Oktober.“  
- **Mehrwert:** Praxisnahes Networking, Einblicke in FHNW‑Projekte, lockere Atmosphäre zum Austausch mit Lehrpersonen und Studierenden.

**Redaktionsplan**  
- **Phase:** Reminder‑Phase (3 Tage vor dem Event).  
- **Wiederkehrende Formate:** Bild‑Post + Kurzinfo (Insta), Text‑Nachricht + Bild (WhatsApp), Bild‑Post + Kurztext (LinkedIn).  
- **Timing:** Veröffentlichung am 13. Oktober, 10 Uhr (optimaler Reichweiten‑Zeitpunkt).  

---

## Instagram  

### Vorschlag 1 – Bild‑Post (Carousel)  
**Kampagnen‑ und Redaktions‑Bezug**  
- Ziel‑Reminder, Kernbotschaft „Gemeinsam feiern“, Mehrwert „Netzwerk‑Boost“.  
- Phase = Reminder, Format = Carousel mit Bild + Infos, Timing = 13. Okt., 10 Uhr.  

**Post‑Text**  
> 📍 Treffpunkt: FHNW‑Campus Olten, Hauptgebäude, Eingang A  

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56a0a349-7f2e-43e5-8d52-5ded467f6e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>